In [1]:
using Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()

using LinearAlgebra, Symbolics, DifferentialEquations, JLD2

  Activating environment at `~/Research/symbolics_double_pendulum/Project.toml`
┌ Info: Precompiling Symbolics [0c5d862f-8b57-4792-8d23-62f2024744c7]
└ @ Base loading.jl:1317
┌ Info: Precompiling DifferentialEquations [0c46a032-eb83-5123-abaf-570d42b7fbaa]
└ @ Base loading.jl:1317


In [2]:
struct DoublePendulum{T}
	m1::T
	m2::T
	l1::T
	l2::T
end

n = 2 # number of states
model = DoublePendulum(1.0, 1.0, 1.0, 1.0) # model

DoublePendulum{Float64}(1.0, 1.0, 1.0, 1.0)

In [3]:
# kinematics
function kinematics_1(model::DoublePendulum, q)
	θ1, θ2 = q

	[0.5 * model.l1 * sin(θ1);
	 -0.5 * model.l1 * cos(θ1)]
end

function kinematics_2(model::DoublePendulum, q)
	θ1, θ2 = q

	[model.l1 * sin(θ1) + 0.5 * model.l2 * sin(θ1 + θ2);
	 -model.l1 * cos(θ1) - 0.5 * model.l2 * cos(θ1 + θ2)]
end

kinematics_2 (generic function with 1 method)

In [4]:
# fast kinematics functions
@variables q[1:n]
@variables q̇[1:n]

k1 = kinematics_1(model, q)
k2 = kinematics_2(model, q)

k1_exp = Symbolics.build_function(k1, q)
k2_exp = Symbolics.build_function(k2, q)

k1_func = eval(k1_exp[1])
k2_func = eval(k2_exp[1])

#7 (generic function with 1 method)

In [5]:
# kinematics Jacobians
j1 = Symbolics.jacobian(k1, q, simplify = true)
j2 = Symbolics.jacobian(k2, q, simplify = true)

j1_exp = Symbolics.build_function(j1, q)
j2_exp = Symbolics.build_function(j2, q)

j1_func = eval(j1_exp[1])
j2_func = eval(j2_exp[1])

#11 (generic function with 1 method)

In [6]:
# Lagrangian
function lagrangian(model, q, q̇)
	L = 0.0

	# mass 1
	v1 = j1_func(q) * q̇
	L += 0.5 * model.m1 * transpose(v1) * v1 		# kinetic energy
	L -= model.m1 * 9.81 * k1_func(q)[2]            # potential energy

	# mass 2
	v2 = j2_func(q) * q̇
	L += 0.5 * model.m2 * transpose(v2) * v2
	L -= model.m2 * 9.81 * k2_func(q)[2]

	return L
end

# fast Lagrangian
L = lagrangian(model, q, q̇)

# 
dLq = Symbolics.gradient(L, q, simplify = true)
dLq̇ = Symbolics.gradient(L, q̇, simplify = true)
ddL = Symbolics.hessian(L, [q; q̇], simplify = true)

# mass matrix
M = ddL[n .+ (1:n), n .+ (1:n)]
M = simplify.(M)

# dynamics bias
C = ddL[n .+ (1:n), 1:n] * q̇ - dLq
C = simplify.(C)

2-element Vector{Num}:
     q̇₂*((cos(q₁) + 0.5cos(q₁ + q₂))*(-0.25q̇₁*sin(q₁ + q₂) - (0.25q̇₂*sin(q₁ + q₂))) + (sin(q₁) + 0.5sin(q₁ + q₂))*(0.25q̇₁*cos(q₁ + q₂) + 0.25q̇₂*cos(q₁ + q₂)) + (0.5cos(q₁) + 0.25cos(q₁ + q₂))*(-0.5q̇₁*sin(q₁ + q₂) - (0.5q̇₂*sin(q₁ + q₂))) + (0.5sin(q₁) + 0.25sin(q₁ + q₂))*(0.5q̇₁*cos(q₁ + q₂) + 0.5q̇₂*cos(q₁ + q₂)) + 0.25cos(q₁ + q₂)*(q̇₁*(sin(q₁) + 0.5sin(q₁ + q₂)) + 0.5q̇₂*sin(q₁ + q₂)) + 0.5cos(q₁ + q₂)*(0.5q̇₁*(sin(q₁) + 0.5sin(q₁ + q₂)) + 0.25q̇₂*sin(q₁ + q₂)) - (0.25sin(q₁ + q₂)*(q̇₁*(cos(q₁) + 0.5cos(q₁ + q₂)) + 0.5q̇₂*cos(q₁ + q₂))) - (0.5sin(q₁ + q₂)*(0.5q̇₁*(cos(q₁) + 0.5cos(q₁ + q₂)) + 0.25q̇₂*cos(q₁ + q₂)))) + 14.715sin(q₁) + 4.905sin(q₁ + q₂)
 q̇₂*(2cos(q₁ + q₂)*(0.25q̇₁*(sin(q₁) + 0.5sin(q₁ + q₂)) - (0.125q̇₁*sin(q₁ + q₂))) + 0.5sin(q₁ + q₂)*(0.25q̇₁*cos(q₁ + q₂) + 0.25q̇₂*cos(q₁ + q₂)) + 0.25sin(q₁ + q₂)*(0.5q̇₁*cos(q₁ + q₂) + 0.5q̇₂*cos(q₁ + q₂)) - (0.25sin(q₁ + q₂)*(q̇₁*(cos(q₁) + 0.5cos(q₁ + q₂)) + 0.5q̇₂*cos(q₁ + q₂))) - (0.5sin(q₁ + q₂)*(

In [19]:
# dynamics
# ẋ = [q̇; M \ (-1.0 * C)]
ẋ = [q̇; M \ (-0.5 * q̇ -1.0 * C)] # joint friction
# ẋ = [q̇; M \ (-1.0 * (q - [π / 10; 0.0]) -1.0 * C)] # spring

ẋ = simplify.(ẋ)

ẋ_exp = Symbolics.build_function(ẋ, q, q̇)
dynamics = eval(ẋ_exp[1])

#22 (generic function with 1 method)

In [20]:
# save dynamics function
path = joinpath(pwd(), "dynamics.jld2")
# @save path ẋ_exp
# @load path ẋ_exp

"/home/taylor/Research/symbolics_double_pendulum/dynamics.jld2"

In [21]:
# DifferentialEquations.jl
function dynamics!(ẋ, x, p, t)
	ẋ .= dynamics(view(x, 1:n), view(x, n .+ (1:n)))
end

# simulate
x0 = [0.5 * π; 0.0; 0.0; 0.0]
tspan = (0.0, 10.0)
dt = 0.01
prob = ODEProblem(dynamics!, x0, tspan);

In [22]:
sol = solve(prob, Tsit5(), adaptive = false, dt = dt);

In [23]:
# MeshCat.jl
include(joinpath(pwd(), "visuals.jl"))
vis = Visualizer()
render(vis)

┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
│ http://127.0.0.1:8704
└ @ MeshCat /home/taylor/.julia/packages/MeshCat/GlCMx/src/visualizer.jl:73


MeshCat.DisplayedVisualizer(MeshCat.CoreVisualizer(MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()), Set{Any}(), ip"127.0.0.1", 8704))

In [24]:
visualize_double_pendulum!(vis, model, sol.u, Δt = dt)